# Challenge Two: Aurora Bay FAQ RAG Chatbot

This notebook implements a Retrieval-Augmented Generation (RAG) chatbot that answers questions about Aurora Bay using BigQuery ML and Vertex AI.

## Key Technologies

- **BigQuery** for storing FAQ data and embeddings
- **BigQuery ML Remote Models** connected to Vertex AI `text-embedding-005`
- **BigQuery VECTOR_SEARCH** for efficient similarity matching
- **Gemini 2.5 Pro** for generating accurate, context-aware answers

## Architecture Overview

1. **Setup**: Create BigQuery connection to Vertex AI and grant permissions
2. **Data Import**: Load Aurora Bay FAQs from GCS (`gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv`) into BigQuery table
3. **Embedding Model**: Create BigQuery ML remote model for `text-embedding-005`
4. **Generate Embeddings**: Use `ML.GENERATE_EMBEDDING` to create embeddings for all FAQ pairs
5. **Vector Search**: User asks question → `VECTOR_SEARCH` finds most similar FAQs by embedding distance
6. **RAG Generation**: Pass retrieved FAQ context + user question to Gemini for final answer

## Workflow

```
User Question
    ↓
ML.GENERATE_EMBEDDING (query embedding)
    ↓
VECTOR_SEARCH (find top-k similar FAQs)
    ↓
Retrieved FAQ Context
    ↓
Gemini 2.5 Pro (generate answer with context)
    ↓
Final Answer
```


## Step 1: Install Dependencies

Install required packages for BigQuery, Vertex AI, and data processing.

In [1]:
pip install --quiet --upgrade google-cloud-bigquery google-cloud-bigquery-connection google-cloud-aiplatform pandas db-dtypes


## Step 2: Import Libraries and Initialize Clients

Import necessary libraries and initialize BigQuery and Vertex AI clients.


In [2]:
import pandas as pd
from google.cloud import bigquery
from google.cloud import aiplatform
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig
from typing import List, Dict

# Configuration
PROJECT_ID = "qwiklabs-gcp-01-752385122246"
LOCATION = "us-central1"
DATASET_ID = "aurora_bay_dataset"
TABLE_ID = "aurora_bay_faqs"
GCS_CSV_PATH = "gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"

# Initialize clients
bq_client = bigquery.Client(project=PROJECT_ID)
vertexai.init(project=PROJECT_ID, location=LOCATION)
aiplatform.init(project=PROJECT_ID, location=LOCATION)

print(f"✓ Initialized BigQuery and Vertex AI for project: {PROJECT_ID}")


✓ Initialized BigQuery and Vertex AI for project: qwiklabs-gcp-01-752385122246


/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


## Step 3: Load CSV from GCS and Create BigQuery Table

Load the Aurora Bay FAQs CSV file from Google Cloud Storage into a pandas DataFrame, then create a BigQuery dataset and table to store the data.


In [3]:
# Load CSV from GCS into pandas
df = pd.read_csv(GCS_CSV_PATH)
print(f"✓ Loaded {len(df)} FAQ entries from GCS")
print(f"\nDataFrame columns: {list(df.columns)}")
print(f"\nFirst few rows:")
print(df.head())

# Create BigQuery dataset if it doesn't exist
dataset_id = f"{PROJECT_ID}.{DATASET_ID}"
try:
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = LOCATION
    dataset = bq_client.create_dataset(dataset, exists_ok=True)
    print(f"\n✓ Dataset {DATASET_ID} ready")
except Exception as e:
    print(f"Dataset creation note: {e}")

# Define schema for the table
schema = [
    bigquery.SchemaField("question", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("answer", "STRING", mode="REQUIRED"),
]

# Create table and load data
table_ref = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"
job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition="WRITE_TRUNCATE",  # Overwrite if exists
)

# Load DataFrame to BigQuery
job = bq_client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()  # Wait for the job to complete

print(f"✓ Loaded {job.output_rows} rows into {table_ref}")

# Verify data loaded correctly
query = f"SELECT COUNT(*) as count FROM `{table_ref}`"
result = bq_client.query(query).result()
for row in result:
    print(f"✓ Verified: {row.count} rows in BigQuery table")


✓ Loaded 50 FAQ entries from GCS

DataFrame columns: ['question', 'answer']

First few rows:
                                         question  \
0                    When was Aurora Bay founded?   
1           What is the population of Aurora Bay?   
2      Where is the Aurora Bay Town Hall located?   
3         Who is the current mayor of Aurora Bay?   
4  What are the primary industries in Aurora Bay?   

                                              answer  
0  Aurora Bay was founded in 1901 by a group of f...  
1  Aurora Bay has a population of approximately 3...  
2  The Town Hall is located at 100 Harbor View Ro...  
3  The current mayor is Linda Greenwood, elected ...  
4  The primary industries include commercial fish...  

✓ Dataset aurora_bay_dataset ready
✓ Loaded 50 rows into qwiklabs-gcp-01-752385122246.aurora_bay_dataset.aurora_bay_faqs
✓ Verified: 50 rows in BigQuery table


## Step 4: Create BigQuery ML Embedding Model

Create a remote model in BigQuery that connects to Vertex AI embeddings for generating vectors.

In [4]:
# Create BigQuery connection for remote models
CONNECTION_ID = "vertex-ai-connection"

from google.cloud import bigquery_connection_v1

def create_bq_connection():
    """Create a BigQuery connection for Vertex AI remote models."""
    try:
        connection_client = bigquery_connection_v1.ConnectionServiceClient()
        parent = f"projects/{PROJECT_ID}/locations/{LOCATION}"

        # Check if connection already exists
        connection_name = f"{parent}/connections/{CONNECTION_ID}"
        try:
            existing = connection_client.get_connection(name=connection_name)
            print(f"✓ Connection already exists: {CONNECTION_ID}")
            return CONNECTION_ID
        except:
            pass  # Connection doesn't exist, create it

        # Create new connection
        connection = bigquery_connection_v1.Connection()
        connection.cloud_resource = bigquery_connection_v1.CloudResourceProperties()

        request = bigquery_connection_v1.CreateConnectionRequest(
            parent=parent,
            connection_id=CONNECTION_ID,
            connection=connection,
        )

        created_connection = connection_client.create_connection(request=request)
        print(f"✓ Created BigQuery connection: {CONNECTION_ID}")
        return CONNECTION_ID

    except Exception as e:
        print(f"Note: {e}")
        print("\nAlternatively, create via command:")
        print(f"!bq mk --connection --location={LOCATION} --project_id={PROJECT_ID} \\")
        print(f"  --connection_type=CLOUD_RESOURCE {CONNECTION_ID}")
        return CONNECTION_ID

# Create or verify connection exists
connection_id = create_bq_connection()
print(f"\nUsing connection: {LOCATION}.{connection_id}")


✓ Connection already exists: vertex-ai-connection

Using connection: us-central1.vertex-ai-connection


In [5]:
# Grant Vertex AI User role to the connection service account
from google.cloud import bigquery_connection_v1

connection_client = bigquery_connection_v1.ConnectionServiceClient()
connection_name = f"projects/{PROJECT_ID}/locations/{LOCATION}/connections/{CONNECTION_ID}"

try:
    connection = connection_client.get_connection(name=connection_name)
    service_account = connection.cloud_resource.service_account_id

    print(f"Connection service account: {service_account}")
    print("\nGranting Vertex AI User role...")

    # Grant the role using gcloud command
    import subprocess

    grant_cmd = [
        "gcloud", "projects", "add-iam-policy-binding", PROJECT_ID,
        f"--member=serviceAccount:{service_account}",
        "--role=roles/aiplatform.user",
        "--condition=None"
    ]

    result = subprocess.run(grant_cmd, capture_output=True, text=True)

    if result.returncode == 0:
        print(f"✓ Granted roles/aiplatform.user to {service_account}")
    else:
        print(f"Note: {result.stderr}")
        print("\nAlternatively, run this command:")
        print(f"!gcloud projects add-iam-policy-binding {PROJECT_ID} \\")
        print(f"  --member=serviceAccount:{service_account} \\")
        print(f"  --role=roles/aiplatform.user")

except Exception as e:
    print(f"Error: {e}")
    print("\nManually grant the Vertex AI User role to the service account shown in the error above.")
    print("Use: https://console.cloud.google.com/iam-admin/iam")


Connection service account: bqcx-11461060539-vixg@gcp-sa-bigquery-condel.iam.gserviceaccount.com

Granting Vertex AI User role...
✓ Granted roles/aiplatform.user to bqcx-11461060539-vixg@gcp-sa-bigquery-condel.iam.gserviceaccount.com


In [6]:
# Create a BigQuery ML remote model that uses Vertex AI text embeddings
embedding_model_id = f"{PROJECT_ID}.{DATASET_ID}.embedding_model"

create_model_sql = f"""
CREATE OR REPLACE MODEL `{embedding_model_id}`
REMOTE WITH CONNECTION `{LOCATION}.{CONNECTION_ID}`
OPTIONS (
  ENDPOINT = 'text-embedding-005'
)
"""

print("Creating BigQuery ML embedding model...")
bq_client.query(create_model_sql).result()
print(f"✓ Created embedding model: {embedding_model_id}")


Creating BigQuery ML embedding model...
✓ Created embedding model: qwiklabs-gcp-01-752385122246.aurora_bay_dataset.embedding_model


## Step 5: Generate Embeddings for FAQs

Use BigQuery ML to generate embeddings for all FAQ entries and store them in a new table.


In [7]:
# Generate embeddings for all FAQs using BigQuery ML
# Combine question and answer for better semantic representation
embedded_table_id = f"{TABLE_ID}_embedded"
embedded_table_ref = f"{PROJECT_ID}.{DATASET_ID}.{embedded_table_id}"

generate_embeddings_sql = f"""
CREATE OR REPLACE TABLE `{embedded_table_ref}` AS
SELECT
  question,
  answer,
  CONCAT(question, ' ', answer) AS content,
  ml_generate_embedding_result
FROM
  ML.GENERATE_EMBEDDING(
    MODEL `{embedding_model_id}`,
    (SELECT question, answer, CONCAT(question, ' ', answer) AS content
     FROM `{table_ref}`)
  )
"""

print("Generating embeddings for all FAQs using BigQuery ML...")
job = bq_client.query(generate_embeddings_sql)
result = job.result()

print(f"✓ Created table with embeddings: {embedded_table_ref}")

# Verify embeddings were generated
verify_query = f"""
SELECT question, ARRAY_LENGTH(ml_generate_embedding_result) as embedding_dim
FROM `{embedded_table_ref}`
LIMIT 5
"""
print("\nVerifying embeddings:")
for row in bq_client.query(verify_query).result():
    print(f"  Question: {row.question[:60]}...")
    print(f"  Embedding dimension: {row.embedding_dim}")
    print()


Generating embeddings for all FAQs using BigQuery ML...
✓ Created table with embeddings: qwiklabs-gcp-01-752385122246.aurora_bay_dataset.aurora_bay_faqs_embedded

Verifying embeddings:
  Question: What is the procedure for trash collection?...
  Embedding dimension: 768

  Question: What broadband or internet services are available?...
  Embedding dimension: 768

  Question: Are there any local newspapers or radio stations?...
  Embedding dimension: 768

  Question: What is the procedure for hosting an event at the waterfront...
  Embedding dimension: 768

  Question: How can I apply for a business license in Aurora Bay?...
  Embedding dimension: 768



## Step 6: Implement Vector Search with VECTOR_SEARCH

Use BigQuery's built-in VECTOR_SEARCH function to find similar FAQs based on embedding similarity.


In [8]:
def search_similar_faqs(user_question: str, top_k: int = 5) -> List[Dict]:
    """
    Search for similar FAQs using BigQuery VECTOR_SEARCH.

    Args:
        user_question: User's query text
        top_k: Number of similar FAQs to return

    Returns:
        List of dictionaries with question, answer, and distance score
    """
    # Use VECTOR_SEARCH with ML.GENERATE_EMBEDDING for the query
    search_sql = f"""
    SELECT
        query.query,
        base.question,
        base.answer,
        distance
    FROM
        VECTOR_SEARCH(
            TABLE `{embedded_table_ref}`,
            'ml_generate_embedding_result',
            (
                SELECT ml_generate_embedding_result, content AS query
                FROM ML.GENERATE_EMBEDDING(
                    MODEL `{embedding_model_id}`,
                    (SELECT '{user_question}' AS content)
                )
            ),
            top_k => {top_k},
            options => '{{"fraction_lists_to_search": 0.01}}'
        )
    """

    results = bq_client.query(search_sql).result()

    # Convert to list of dicts
    similar_faqs = []
    for row in results:
        similar_faqs.append({
            "question": row.question,
            "answer": row.answer,
            "distance": row.distance,
            "similarity": 1 - row.distance  # Convert distance to similarity
        })

    return similar_faqs

# Test the search function
test_query = "What are the operating hours?"
print(f"Test search for: '{test_query}'\n")
results = search_similar_faqs(test_query, top_k=5)
for i, result in enumerate(results, 1):
    print(f"{i}. Distance: {result['distance']:.4f} | Similarity: {result['similarity']:.4f}")
    print(f"   Q: {result['question']}")
    print(f"   A: {result['answer'][:100]}...")
    print()


Test search for: 'What are the operating hours?'

1. Distance: 0.9096 | Similarity: 0.0904
   Q: What are the operating hours of the Aurora Bay Public Library?
   A: The library is open Monday through Friday from 9 AM to 6 PM, and on Saturdays from 10 AM to 4 PM. It...

2. Distance: 0.9317 | Similarity: 0.0683
   Q: When does the local fishermen’s market usually take place?
   A: The fishermen’s market runs every Saturday from May through September at the Harborfront area, from ...

3. Distance: 0.9328 | Similarity: 0.0672
   Q: Are there specific quiet hours or noise ordinances?
   A: Yes. Residential noise ordinances go into effect from 10 PM to 6 AM on weekdays and from 11 PM to 7 ...



## Step 7: Build RAG Chatbot

Implement the RAG pipeline with Gemini.

In [9]:
# Initialize Gemini model for answering questions
gemini_model = GenerativeModel("gemini-2.5-pro")

def answer_question(user_question: str, top_k: int = 5, verbose: bool = True) -> str:
    """
    Answer a user question using RAG with Aurora Bay FAQ data.

    Args:
        user_question: The user's question about Aurora Bay
        top_k: Number of similar FAQs to retrieve for context
        verbose: Print retrieved context if True

    Returns:
        Generated answer from Gemini based on retrieved FAQ context
    """
    # Step 1: Search for similar FAQs
    similar_faqs = search_similar_faqs(user_question, top_k=top_k)

    if verbose:
        print(f"Retrieved {len(similar_faqs)} similar FAQs:\n")
        for i, faq in enumerate(similar_faqs, 1):
            print(f"{i}. [{faq['similarity']:.3f}] {faq['question']}")
        print()

    # Step 2: Build context from retrieved FAQs
    context_parts = []
    for i, faq in enumerate(similar_faqs, 1):
        context_parts.append(f"FAQ {i}:")
        context_parts.append(f"Q: {faq['question']}")
        context_parts.append(f"A: {faq['answer']}")
        context_parts.append("")  # Blank line

    context = "\n".join(context_parts)

    # Step 3: Build prompt for Gemini
    prompt = f"""You are a helpful assistant for Aurora Bay. Answer the user's question based ONLY on the provided FAQ context.
If the FAQ context doesn't contain relevant information to answer the question, politely say you don't have that information.

FAQ Context:
{context}

User Question: {user_question}

Answer: Provide a clear, accurate answer based on the FAQ context above."""

    # Step 4: Generate answer with Gemini
    response = gemini_model.generate_content(
        prompt,
        generation_config=GenerationConfig(
            temperature=0.3,  # Lower temperature for more factual responses
            top_p=0.95,
            max_output_tokens=1024,
        )
    )

    return response.text

# Test the RAG chatbot
test_question = "What are the operating hours of Aurora Bay?"
print(f"User Question: {test_question}")
print("="*60)
answer = answer_question(test_question, verbose=True)
print("="*60)
print(f"Answer:\n{answer}")


User Question: What are the operating hours of Aurora Bay?


/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


Retrieved 3 similar FAQs:

1. [0.325] What are the operating hours of the Aurora Bay Public Library?
2. [0.278] Does Aurora Bay have public transportation?
3. [0.250] What is the average temperature range in Aurora Bay?

Answer:
Based on the information provided, here are the operating hours for specific services in Aurora Bay:

*   **The Aurora Bay Public Library** is open Monday through Friday from 9 AM to 6 PM


## Step 8: Demo - Test the RAG Chatbot

Test the chatbot with various Aurora Bay-related questions to demonstrate its capabilities.


In [11]:
# Sample questions to test the chatbot
test_questions = [
    "What is the capital of France?",  # Irrelevant - should return "no information"
    "What are the operating hours of the Aurora Bay Public Library?",
    "What is the average temperature range in Aurora Bay?",
    "Does Aurora Bay have public transportation?",
    "What attractions and activities can I enjoy at Aurora Bay?",
]

print("="*70)
print("AURORA BAY FAQ CHATBOT SIM")
print("="*70)

for question in test_questions:
    print(f"\n{'─'*70}")
    print(f"Question: {question}")
    print('─'*70)

    try:
        answer = answer_question(question, top_k=5, verbose=False)
        print(f"\nAnswer:\n{answer}")
    except Exception as e:
        print(f"Error: {e}")

    print()

print("="*70)
print("Sim complete!")
print("="*70)


AURORA BAY FAQ CHATBOT SIM

──────────────────────────────────────────────────────────────────────
Question: What is the capital of France?
──────────────────────────────────────────────────────────────────────

Answer:
I don't have that information.


──────────────────────────────────────────────────────────────────────
Question: What are the operating hours of the Aurora Bay Public Library?
──────────────────────────────────────────────────────────────────────

Answer:
The Aurora Bay Public Library is open Monday through Friday from 9 AM to 6 PM, and on Saturdays from 10 AM to 4 PM. It is closed on Sundays and major holidays.


──────────────────────────────────────────────────────────────────────
Question: What is the average temperature range in Aurora Bay?
──────────────────────────────────────────────────────────────────────

Answer:
Winters in Aurora Bay average between 10°F to 25°F, while summers are milder, around 50°F to 65°F.


──────────────────────────────────────────────